<center>
    <img src="https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/Logos/organization_logo/organization_logo.png" width="300" alt="cognitiveclass.ai logo"  />
</center>

# Connect to Db2 database on Cloud using Python

Estimated time needed: **15** minutes

## Objectives

After completing this lab you will be able to:

*   Import the ibm_db Python library
*   Enter the database connection credentials
*   Create the database connection
*   Close the database connection


**Note:** Please follow the instructions given in the first Lab of this course to Create a database service instance of Db2 on Cloud and retrieve your database Service Credentials.

## Import the `ibm_db` Python library

The `ibm_db` [API ](https://pypi.python.org/pypi/ibm_db/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDB0201ENSkillsNetwork20127838-2021-01-01) provides a variety of useful Python functions for accessing and manipulating data in an IBM® data server database, including functions for connecting to a database, preparing and issuing SQL statements, fetching rows from result sets, calling stored procedures, committing and rolling back transactions, handling errors, and retrieving metadata.

We first import the ibm_db library into our Python Application

Execute the following cells by clicking within it and then
press `Shift` and `Enter` keys simultaneously

The following required modules are pre-installed in the Skills Network Labs environment. However if you run this notebook commands in a different Jupyter environment (e.g. Watson Studio or Ananconda) you may need to install these libraries by removing the `#` sign before `!pip` in the code cell below.


In [1]:
# These libraries are pre-installed in SN Labs. If running in another environment please uncomment lines below to install them:
!pip install --force-reinstall ibm_db==3.1.0 ibm_db_sa==0.3.3
# Ensure we don't load_ext with sqlalchemy>=1.4 (incompadible)
!pip uninstall sqlalchemy==1.4 -y && pip install sqlalchemy==1.3.24
!pip install ipython-sql

  Using cached ibm_db-3.1.0-py3-none-any.whl
  Using cached ibm_db_sa-0.3.3.tar.gz (24 kB)
  Using cached SQLAlchemy-1.4.31-cp37-cp37m-win_amd64.whl (1.5 MB)
  Using cached greenlet-1.1.2-cp37-cp37m-win_amd64.whl (101 kB)
  Using cached importlib_metadata-4.11.1-py3-none-any.whl (17 kB)
  Using cached typing_extensions-4.1.1-py3-none-any.whl (26 kB)
  Using cached zipp-3.7.0-py3-none-any.whl (5.3 kB)
Using legacy 'setup.py install' for ibm-db-sa, since package 'wheel' is not installed.
  Attempting uninstall: zipp
    Found existing installation: zipp 3.7.0
    Uninstalling zipp-3.7.0:
      Successfully uninstalled zipp-3.7.0
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 4.1.1
    Uninstalling typing-extensions-4.1.1:
      Successfully uninstalled typing-extensions-4.1.1
  Attempting uninstall: importlib-metadata
    Found existing installation: importlib-metadata 4.11.1
    Uninstalling importlib-metadata-4.11.1:
      Successfully unin

ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'c:\\users\\charwang\\pycharmprojects\\baidu\\venv\\lib\\site-packages\\~qlalchemy\\cprocessors.cp37-win_amd64.pyd'
Check the permissions.

You should consider upgrading via the 'c:\users\charwang\pycharmprojects\baidu\venv\scripts\python.exe -m pip install --upgrade pip' command.


Found existing installation: SQLAlchemy 1.4.31

You should consider upgrading via the 'c:\users\charwang\pycharmprojects\baidu\venv\scripts\python.exe -m pip install --upgrade pip' command.



Uninstalling SQLAlchemy-1.4.31:
  Successfully uninstalled SQLAlchemy-1.4.31
  Using cached SQLAlchemy-1.3.24-cp37-cp37m-win_amd64.whl (1.2 MB)


You should consider upgrading via the 'c:\users\charwang\pycharmprojects\baidu\venv\scripts\python.exe -m pip install --upgrade pip' command.


In [1]:
import ibm_db

When the command above completes, the `ibm_db` library is loaded in your notebook.

## Identify the database connection credentials

Connecting to dashDB or DB2 database requires the following information:

*   Driver Name
*   Database name
*   Host DNS name or IP address
*   Host port
*   Connection protocol
*   User ID (or username)
*   User Password

**Notice:** To obtain credentials please refer to the instructions given in the first Lab of this course

Now enter your database credentials below and execute the cell with `Shift` + `Enter`


In [2]:
#Replace the placeholder values with your actual Db2 hostname, username, and password:
dsn_hostname = "" # e.g.: "54a2f15b-5c0f-46df-8954-7e38e612c2bd.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud"
dsn_uid = ""        # e.g. "abc12345"
dsn_pwd = ""      # e.g. "7dBZ3wWt9XN6$o0J"

dsn_driver = "{IBM DB2 ODBC DRIVER}"
dsn_database = "BLUDB"            # e.g. "BLUDB"
dsn_port = "30376"                # e.g. "32733" 
dsn_protocol = "TCPIP"            # i.e. "TCPIP"
dsn_security = "SSL"              #i.e. "SSL"

## Create the DB2 database connection

Ibm_db API uses the IBM Data Server Driver for ODBC and CLI APIs to connect to IBM DB2 and Informix.

Lets build the dsn connection string using the credentials you entered above


In [3]:
#DO NOT MODIFY THIS CELL. Just RUN it with Shift + Enter
#Create the dsn connection string
dsn = (
    "DRIVER={0};"
    "DATABASE={1};"
    "HOSTNAME={2};"
    "PORT={3};"
    "PROTOCOL={4};"
    "UID={5};"
    "PWD={6};"
    "SECURITY={7};").format(dsn_driver, dsn_database, dsn_hostname, dsn_port, dsn_protocol, dsn_uid, dsn_pwd,dsn_security)

#print the connection string to check correct values are specified
print(dsn)

DRIVER={IBM DB2 ODBC DRIVER};DATABASE=BLUDB;HOSTNAME=6667d8e9-9d4d-4ccb-ba32-21da3bb5aafc.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud;PORT=30376;PROTOCOL=TCPIP;UID=sny28808;PWD=tat7wpoOVV27A2h4;SECURITY=SSL;


Now establish the connection to the database


In [4]:
#DO NOT MODIFY THIS CELL. Just RUN it with Shift + Enter
#Create database connection

try:
    conn = ibm_db.connect(dsn, "", "")
    print ("Connected to database: ", dsn_database, "as user: ", dsn_uid, "on host: ", dsn_hostname)

except:
    print ("Unable to connect: ", ibm_db.conn_errormsg() )


Connected to database:  BLUDB as user:  sny28808 on host:  6667d8e9-9d4d-4ccb-ba32-21da3bb5aafc.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud


Congratulations if you were able to connect successfuly. Otherwise check the error and try again.


In [5]:
import pandas
import ibm_db_dbi

#connection for pandas
pconn = ibm_db_dbi.Connection(conn)

In [6]:
# Q1:Find the total number of crimes recorded in the CRIME table.

selectQuery = "select count(*) from CHICAGO_CRIME_DATA"
pdf = pandas.read_sql(selectQuery, pconn)
pdf

,1
0,533


In [7]:
# Q2:List community areas with per capita income less than 11000.

selectQuery = "select COMMUNITY_AREA_NUMBER,COMMUNITY_AREA_NAME from CENSUS_DATA where PER_CAPITA_INCOME < 11000"
pdf = pandas.read_sql(selectQuery, pconn)
pdf

,COMMUNITY_AREA_NUMBER,COMMUNITY_AREA_NAME
0,26,West Garfield Park
1,30,South Lawndale
2,37,Fuller Park
3,54,Riverdale


In [15]:
# Q3: List all case numbers for crimes involving minors?

selectQuery = "select CASE_NUMBER,DESCRIPTION from CHICAGO_CRIME_DATA where UPPER(DESCRIPTION) like '%MINOR%'"
pdf = pandas.read_sql(selectQuery, pconn)
pdf

,CASE_NUMBER,DESCRIPTION
0,HL266884,SELL/GIVE/DEL LIQUOR TO MINOR
1,HK238408,ILLEGAL CONSUMPTION BY MINOR


In [16]:
# Q4: List all kidnapping crimes involving a child?(children are not considered minors for the purposes of crime analysis)

selectQuery = "select CASE_NUMBER, PRIMARY_TYPE, DESCRIPTION from CHICAGO_CRIME_DATA where PRIMARY_TYPE = 'KIDNAPPING'"
pdf = pandas.read_sql(selectQuery, pconn)
pdf

,CASE_NUMBER,PRIMARY_TYPE,DESCRIPTION
0,HN144152,KIDNAPPING,CHILD ABDUCTION/STRANGER


In [17]:
# Q5: What kind of crimes were recorded at schools?

selectQuery = "select DISTINCT(PRIMARY_TYPE) from CHICAGO_CRIME_DATA where LOCATION_DESCRIPTION like '%SCHOOL%'"
pdf = pandas.read_sql(selectQuery, pconn)
pdf

,PRIMARY_TYPE
0,ASSAULT
1,BATTERY
2,CRIMINAL DAMAGE
3,CRIMINAL TRESPASS
4,NARCOTICS
5,PUBLIC PEACE VIOLATION


In [6]:
# Q6: List the average safety score for all types of schools.
# change column "Elementary, Middle, or High School" to "School_Type" when creating table and loading the data

selectQuery = "select SCHOOL_TYPE,AVG(SAFETY_SCORE) from CHICAGO_PUBLIC_SCHOOLS group by SCHOOL_TYPE"
pdf = pandas.read_sql(selectQuery, pconn)
pdf

,SCHOOL_TYPE,2
0,ES,49
1,HS,49
2,MS,48


In [10]:
# Q7：List 5 community areas with highest % of households below poverty line

selectQuery = "select COMMUNITY_AREA_NAME, PERCENT_HOUSEHOLDS_BELOW_POVERTY from CENSUS_DATA \
ORDER BY PERCENT_HOUSEHOLDS_BELOW_POVERTY DESC LIMIT 5"
pdf = pandas.read_sql(selectQuery, pconn)
pdf

,COMMUNITY_AREA_NAME,PERCENT_HOUSEHOLDS_BELOW_POVERTY
0,Riverdale,56.5
1,Fuller Park,51.2
2,Englewood,46.6
3,North Lawndale,43.1
4,East Garfield Park,42.4


In [13]:
# Q8: Which community area(number) is most crime prone

selectQuery = "select COMMUNITY_AREA_NUMBER, count(*) as CRIME_COUNT from CHICAGO_CRIME_DATA \
group by COMMUNITY_AREA_NUMBER order by count(*) desc limit 1"
pdf = pandas.read_sql(selectQuery, pconn)
pdf

,COMMUNITY_AREA_NUMBER,CRIME_COUNT
0,25,43


In [15]:
# Q9: Use a sub-query to find the name of the community area with highest hardship index

selectQuery = "select COMMUNITY_AREA_NAME, HARDSHIP_INDEX from CENSUS_DATA \
where HARDSHIP_INDEX = (select max(HARDSHIP_INDEX) from CENSUS_DATA)"
pdf = pandas.read_sql(selectQuery, pconn)
pdf

,COMMUNITY_AREA_NAME,HARDSHIP_INDEX
0,Riverdale,98


In [29]:
# Q10: Use a sub-query to determine the Community Area Name with most number of crimes?

selectQuery = "select COMMUNITY_AREA_NAME from CENSUS_DATA \
where COMMUNITY_AREA_NUMBER = \
(select COMMUNITY_AREA_NUMBER from CHICAGO_CRIME_DATA \
group by COMMUNITY_AREA_NUMBER \
order by count(*) desc \
limit 1) "
pdf = pandas.read_sql(selectQuery, pconn)
pdf

,COMMUNITY_AREA_NAME
0,Austin


## Close the Connection

We free all resources by closing the connection. Remember that it is always important to close connections so that we can avoid unused connections taking up resources.


In [8]:
ibm_db.close(conn)

True

## Summary

In this tutorial you established a connection to a DB2 database on Cloud database from a Python notebook using ibm_db API.


## Author

<a href="https://www.linkedin.com/in/ravahuja?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDB0201ENSkillsNetwork20127838-2021-01-01">Rav Ahuja</a>

## Change Log

| Date (YYYY-MM-DD) | Version | Changed By | Change Description                 |
| ----------------- | ------- | ---------- | ---------------------------------- |
| 2021-11-17        | 2.2     | Lakshmi    | Updated library                    |
| 2021-07-09        | 2.1     | Malika     | Updated the connection string      |
| 2020-08-28        | 2.0     | Lavanya    | Moved lab to course repo in GitLab |

<hr>

## <h3 align="center"> © IBM Corporation 2020. All rights reserved. <h3/>
